In [1]:
from datetime import date

from create_data import recreate_databases, create_tables
from jobs.enrich_action import EnrichCEOrderCreatedJob
from utils import get_spark_session

In [2]:
spark = get_spark_session()

2021-11-26 16:23:51.606 | INFO     | utils:get_spark_session:7 - Creating Spark Session


In [3]:
recreate_databases(spark)

2021-11-26 16:24:01.817 | INFO     | create_data:recreate_databases:29 - Removing spark warehouse (path = 'spark-warehouse')
2021-11-26 16:24:01.875 | INFO     | create_data:recreate_databases:33 - Creating database 'standardized_glovo_live'
2021-11-26 16:24:05.480 | INFO     | create_data:recreate_databases:33 - Creating database 'mpcustomer_custom_events'
2021-11-26 16:24:05.544 | INFO     | create_data:recreate_databases:33 - Creating database 'mpcustomer_screen_views'
2021-11-26 16:24:05.601 | INFO     | create_data:recreate_databases:33 - Creating database 'enriched_custom_events'
2021-11-26 16:24:05.640 | INFO     | create_data:recreate_databases:33 - Creating database 'enriched_screen_views'


In [4]:
create_tables(spark)

2021-11-26 16:24:05.692 | INFO     | create_data:create_table:91 - Creating table 'standardized_glovo_live.cities'
2021-11-26 16:24:17.300 | INFO     | create_data:create_table:91 - Creating table 'standardized_glovo_live.devices'
2021-11-26 16:24:27.103 | INFO     | create_data:create_table:91 - Creating table 'standardized_glovo_live.orders'
2021-11-26 16:24:39.936 | INFO     | create_data:create_table:91 - Creating table 'standardized_glovo_live.customer_subscriptions'
2021-11-26 16:24:50.309 | INFO     | create_data:create_table:91 - Creating table 'standardized_glovo_live.customer_subscription_periods'
2021-11-26 16:24:57.993 | INFO     | create_data:create_table:91 - Creating table 'mpcustomer_custom_events.order_created'


In [ ]:
order_created_job = EnrichCEOrderCreatedJob(
    spark=spark,
    exec_date=date(2021, 11, 19),
    n_days=3
)
order_created_job.run()

2021-11-26 16:25:05.598 | INFO     | loaders:load:47 - Loading 'mpcustomer_custom_events.order_created'
2021-11-26 16:25:05.753 | INFO     | loaders:load:47 - Loading 'standardized_glovo_live.cities'
2021-11-26 16:25:05.873 | INFO     | loaders:load:47 - Loading 'standardized_glovo_live.orders'
2021-11-26 16:25:05.911 | INFO     | ports.live_db:load:72 - Loading 'standardized_glovo_live.customer_subscriptions'
2021-11-26 16:25:06.100 | INFO     | transformations.interfaces:_apply:13 - Applying transformation 'AddTimezone'
2021-11-26 16:25:06.147 | INFO     | transformations.interfaces:_apply:13 - Applying transformation 'AddNumberOfOrders'
2021-11-26 16:25:06.540 | INFO     | transformations.interfaces:_apply:13 - Applying transformation 'AddIsPrime'
2021-11-26 16:25:06.700 | INFO     | writers:write:23 - Writting 'enriched_custom_events.order_created'


# Show input

In [ ]:
spark.table("mpcustomer_custom_events.order_created").show()

# Show outputs

In [ ]:
spark.table("enriched_custom_events.order_created").show()